In [3]:
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import string 
import operator
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
# Load the LDA model from sk-learn
from sklearn.decomposition import LatentDirichletAllocation as LDA

In [4]:
barley=pd.read_csv('../Dataset/barley_no_zero_duplicates.csv')
#bean=pd.read_csv('bean_no_zero_duplicates.csv')
corn_maize=pd.read_csv('../Dataset/corn_maize_no_zero_duplicates.csv')
#edamame=pd.read_csv('edamame_no_zero_duplicates.csv')
#lentil=pd.read_csv('lentil_no_zero_duplicates.csv')
millet=pd.read_csv('../Dataset/millet_no_zero_duplicates.csv')
#nut=pd.read_csv('nut_no_zero_duplicates.csv')
oat=pd.read_csv('../Dataset/oat_no_zero_duplicates.csv')
#pea=pd.read_csv('pea_no_zero_duplicates.csv')
potato=pd.read_csv('../Dataset/potato_no_zero_duplicates.csv')
quinoa=pd.read_csv('../Dataset/quinoa_no_zero_duplicates.csv')
rice=pd.read_csv('../Dataset/rice_no_zero_duplicates.csv')
rye=pd.read_csv('../Dataset/rye_no_zero_duplicates.csv')
sorghum=pd.read_csv('../Dataset/sorghum_no_zero_duplicates.csv')
soybean=pd.read_csv('../Dataset/soybean_no_zero_duplicates.csv')
wheat=pd.read_csv('../Dataset/wheat_no_zero_duplicates.csv')
legume_nuts_no_peanuts_no_zero_duplicates=pd.read_csv('../Dataset/legume_no_zero_duplicates.csv')
legume_peanuts_no_zero_duplicates=pd.read_csv('../Dataset/legume_peanuts_no_zero_duplicates.csv')
legume_other=pd.read_csv('../Dataset/legume_other_no_zero_duplicates.csv')
whole=pd.read_csv('../Dataset/all_no_zero_duplicates.csv')

# Combined analysis for all the crops

In [5]:
wheat.head()

,PROJECT_ID,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,...,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ABSTRACT
0,584433,Anabolism; antiporter; Arabidopsis; Biochemis...,"PHYTONUTRIENT BIOCHEMISTRY, PHYSIOLOGY, AND TR...",USDA,ARS,NaN,ARS-0408087,5/21/2004,3/31/2009,"UPCHURCH, DAN R",...,TX,77030,UNITED STATES,NaN,NaN,10.001,2004,NaN,NaN,Objective(s): 1) Identify and characterize pla...
1,584463,Affect; Biochemical; Bread; Breeding; Carbohy...,ENHANCE HARD SPRING AND DURUM WHEAT QUALITY AN...,USDA,ARS,NaN,ARS-0408465,9/10/2004,7/5/2009,"HARELAND, GARY A",...,ND,58102-2765,UNITED STATES,NaN,NaN,10.001,2004,NaN,NaN,Objective(s): Evaluate the processing and end-...
2,584455,Avena sativa; biosecurity; Breeding; Cereals;...,OBJECTIVE GRADING AND END-USE PROPERTY ASSESSM...,USDA,ARS,NaN,ARS-0408432,9/25/2004,7/5/2009,"PEARSON, THOMAS CLARK",...,KS,66502,UNITED STATES,NaN,NaN,10.001,2004,NaN,NaN,Objective(s): Develop instrumentation and proc...
3,584464,Amino Acids; Amylopectin; Amylose; base; Bioc...,OPTICAL PROPERTIES OF SMALL GRAINS FOR RAPID A...,USDA,ARS,NaN,ARS-0408471,9/15/2004,9/3/2009,"DELWICHE, STEPHEN R",...,MD,20705,UNITED STATES,NaN,NaN,10.001,2004,NaN,NaN,Objective(s): (1) Develop an automated system ...
4,584523,Architecture; Bioinformatics; Cell Wall; Cere...,BIOTECHNOLOGICAL ENHANCEMENT OF ENERGY CROPS,USDA,ARS,NaN,ARS-0408875,8/15/2004,8/14/2009,"ANDERSON, OLIN D",...,CA,94710,UNITED STATES,NaN,NaN,10.001,2004,NaN,NaN,Objective(s): Develop the necessary range of k...


In [6]:
wheat.shape

(1319, 25)

In [7]:
data=[barley,corn_maize,millet,oat,legume,potato,quinoa,rice,rye,sorghum,soybean,wheat,whole]
category=['barley','corn_maize','millet','oat','legume','potato','quinoa','rice','rye',
         'sorghum','soybean','wheat','all']

In [8]:
# Helper function
def plot_10_most_common_words(count_data, count_vectorizer,filename):
    import matplotlib.pyplot as plt
    words = count_vectorizer.get_feature_names()
    total_counts = np.zeros(len(words))
    for t in count_data:
        total_counts+=t.toarray()[0]
    
    count_dict = (zip(words, total_counts))
    count_dict = sorted(count_dict, key=lambda x:x[1], reverse=True)[0:10]
    words = [w[0] for w in count_dict]
    counts = [w[1] for w in count_dict]
    x_pos = np.arange(len(words)) 
    
    fig=plt.figure(2, figsize=(15, 15/1.6180))
    plt.subplot(title='10 most common words')
    sns.set_context("notebook", font_scale=1.25, rc={"lines.linewidth": 2.5})
    sns.barplot(x_pos, counts, palette='husl')
    plt.xticks(x_pos, words, rotation=90) 
    plt.xlabel('words')
    plt.ylabel('counts')
    plt.title(filename)
    #plt.show()
    temp='topic_modelling/'+filename+'.png'
    plt.savefig(temp)
    plt.close(fig)

In [13]:
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx+", ".join([words[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [14]:
def preprocess(data,category):
    d = dict() 
    df=pd.DataFrame()
    text=''
    funding=[]
    # word-count in csv
    i=0
    for dataframe in data:
        print(category[i])
        dataframe['ABSTRACT']=dataframe['ABSTRACT'].astype(str)
        dataframe['ABSTRACT_PROCESSED'] = dataframe['ABSTRACT'].map(lambda x: re.sub('[,\.!?]', '', x))
        # Convert the titles to lowercase
        dataframe['ABSTRACT_PROCESSED'] = dataframe['ABSTRACT_PROCESSED'].map(lambda x: x.lower())
        # Print out the first rows of papers
        #dataframe['ABSTRACT_PROCESSED'].head()
        # Initialise the count vectorizer with the English stop words
        count_vectorizer = CountVectorizer(stop_words='english')
        # Fit and transform the processed titles
        count_data = count_vectorizer.fit_transform(dataframe['ABSTRACT_PROCESSED'])
        # Visualise the 10 most common words
        plot_10_most_common_words(count_data, count_vectorizer,category[i])
        number_topics = 10
        number_words = 10
        # Create and fit the LDA model
        lda = LDA(n_components=number_topics, n_jobs=-1)
        lda.fit(count_data)
        # Print the topics found by the LDA model
        print("Topics found via LDA:")
        print_topics(lda, count_vectorizer, number_words)
        i=i+1
        print('-------')
        print('-------')
        
               
preprocess(data,category)

barley
Topics found via LDA:
Topic #0:objective, barley, wheat, rust, develop, germplasm, resistance, identify, use, sub
Topic #1:crops, crop, barley, production, wheat, new, research, resistance, management, project
Topic #2:grain, wheat, barley, develop, crop, systems, production, quality, yield, plant
Topic #3:ipm, pathogen, pest, program, barley, crops, species, pathogens, wheat, management
Topic #4:barley, project, grain, mycotoxins, production, north, research, agriculture, dakota, food
Topic #5:barley, objective, research, fusarium, yield, maize, blight, wheat, head, molecular
Topic #6:genes, plant, maize, gene, research, important, rice, project, barley, genome
Topic #7:resistance, wheat, genes, objective, identify, barley, develop, aphid, determine, sub
Topic #8:plant, host, virus, viruses, disease, project, viral, research, plants, high
Topic #9:flowering, energy, biomass, process, coal, barley, time, production, research, fuel
-------
-------
corn_maize
Topics found via LDA:

Topics found via LDA:
Topic #0:plant, genes, project, gene, genome, crop, sorghum, research, maize, rice
Topic #1:sorghum, crop, corn, soil, management, grain, production, crops, use, water
Topic #2:corn, sorghum, food, grain, new, project, use, research, high, pest
Topic #3:sorghum, objective, germplasm, genetic, identify, resistance, subobjective, stress, drought, develop
Topic #4:insect, plant, objective, pest, pests, maize, crops, sub, sorghum, resistance
Topic #5:crops, nematodes, tolerance, al, nematode, research, rice, root, production, growth
Topic #6:sorghum, genetic, wheat, aphid, objective, breeding, resistance, genes, plant, develop
Topic #7:sorghum, soil, water, effects, determine, lignin, carbon, production, starch, bioenergy
Topic #8:production, crop, water, crops, systems, sorghum, research, use, management, biomass
Topic #9:sorghum, production, develop, objective, technologies, sugar, enable, products, commercial, sweet
-------
-------
soybean
Topics found via LDA:
Top